# PROCESO DE ETL

## Extraccion de datos

In [3]:
#Importamos las librerias necesarias

import pandas as pd
import numpy as np

In [2]:
# Abrimos los CSV con pandas para su posterior tratamiento

amazon = pd.read_csv('./MLOpsReviews/amazon_prime_titles.csv', sep= ',')
disney = pd.read_csv('./MLOpsReviews/disney_plus_titles.csv', sep= ',')
hulu = pd.read_csv('./MLOpsReviews/hulu_titles.csv', sep= ',')
netflix = pd.read_csv('./MLOpsReviews/netflix_titles.csv', sep=',')

### Visualización de los datasets

In [3]:
amazon.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...


In [5]:
disney.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!


In [6]:
hulu.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...


In [7]:
netflix.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."


## Transformaciones

### Generamos la nueva columna 'id'

In [4]:
# Generamos el nuevo id
 
amazon['id'] = 'a' + amazon['show_id']
disney['id'] = 'd' + disney['show_id']
hulu['id'] = 'h' + hulu['show_id']
netflix['id'] = 'n' + netflix['show_id']

# Visualizo para corroborar que los cambios hayan sido correctamente aplicados

amazon['id']
disney['id']
hulu['id']
netflix['id']

0          ns1
1          ns2
2          ns3
3          ns4
4          ns5
         ...  
8802    ns8803
8803    ns8804
8804    ns8805
8805    ns8806
8806    ns8807
Name: id, Length: 8807, dtype: object

In [7]:
# Generamos una nueva columna con el nombre de la plataforma

amazon['platform'] = 'amazon'
disney['platform'] = 'disney'
hulu['platform'] = 'hulu'
netflix['platform'] = 'netflix'

In [8]:
# Unifico los cuatro datasets en uno

df_completo = pd.concat([amazon, disney, hulu, netflix])

In [9]:
#Reemplazo los valores nulos de la columna rating por G (general for all audiences)

df_completo['rating'].fillna('G',inplace=True)

#Vizualizo para corroborar que los cambios hayan sido correctamente aplicados

df_completo['rating']

0           G
1         13+
2           G
3           G
4           G
        ...  
8802        R
8803    TV-Y7
8804        R
8805       PG
8806    TV-14
Name: rating, Length: 22998, dtype: object

### Estandarización del formato de fecha a AAAA-mm-dd

In [10]:
df_completo['date_added'] = pd.to_datetime(df_completo['date_added'])

#Vizualizo para corroborar que los cambios hayan sido correctamente aplicados

df_completo['date_added']


0      2021-03-30
1      2021-03-30
2      2021-03-30
3      2021-03-30
4      2021-03-30
          ...    
8802   2019-11-20
8803   2019-07-01
8804   2019-11-01
8805   2020-01-11
8806   2019-03-02
Name: date_added, Length: 22998, dtype: datetime64[ns]

In [11]:
#Convierto los campos de texto en minúsculas

df_completo = df_completo.apply(lambda x: x.str.lower() if x.dtype == "object" else x)

### Division del campo duration en 'duration_int' y 'duration_type'

In [12]:
df_completo['duration'] = df_completo['duration'].str.replace(r'', '')

df_completo[['duration_int','duration_type']] = df_completo['duration'].str.split(expand=True)


# Cambiamos el tipo de dato de 'duration_int' a int

df_completo['duration_int'] = df_completo['duration_int'].astype('Int64')

#Reemplazo "seasons" por "season":

df_completo["duration_type"] = df_completo["duration_type"].str.replace("seasons", "season")

In [13]:
df_completo.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,platform,duration_int,duration_type
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,as1,amazon,113,min
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,as2,amazon,110,min
2,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74 min,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3,amazon,74,min
3,s4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,69 min,documentary,"pink breaks the mold once again, bringing her ...",as4,amazon,69,min
4,s5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,45 min,"drama, fantasy",teenage matt banting wants to work with a famo...,as5,amazon,45,min


In [14]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 8806
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   show_id        22998 non-null  object        
 1   type           22998 non-null  object        
 2   title          22998 non-null  object        
 3   director       14739 non-null  object        
 4   cast           17677 non-null  object        
 5   country        11499 non-null  object        
 6   date_added     13444 non-null  datetime64[ns]
 7   release_year   22998 non-null  int64         
 8   rating         22998 non-null  object        
 9   duration       22516 non-null  object        
 10  listed_in      22998 non-null  object        
 11  description    22994 non-null  object        
 12  id             22998 non-null  object        
 13  platform       22998 non-null  object        
 14  duration_int   22516 non-null  Int64         
 15  duration_type  22516

## Guardamos nuestro nuevo dataset

In [15]:
df_completo.to_csv("datasets/movies_titles.csv", index=False)

## ETL de los ratings

In [18]:
df_1 = pd.read_csv('MLOpsReviews/ratings/1.csv')
df_2 = pd.read_csv('MLOpsReviews/ratings/2.csv')
df_3 = pd.read_csv('MLOpsReviews/ratings/3.csv')
df_4 = pd.read_csv('MLOpsReviews/ratings/4.csv')
df_5 = pd.read_csv('MLOpsReviews/ratings/5.csv')
df_6 = pd.read_csv('MLOpsReviews/ratings/6.csv')
df_7 = pd.read_csv('MLOpsReviews/ratings/7.csv')
df_8 = pd.read_csv('MLOpsReviews/ratings/8.csv')

In [19]:
df_ratings = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8])

### Transformamos la columna de timestamps 

In [20]:
import datetime

In [21]:
df_ratings['timestamp']

0          1425941529
1          1425942435
2          1425941523
3          1425941546
4          1425941556
              ...    
1499995    1196786159
1499996    1196786030
1499997    1196785679
1499998    1196787089
1499999    1196785847
Name: timestamp, Length: 11024289, dtype: int64

In [22]:
df_ratings['timestamp'] = pd.to_datetime(df_ratings['timestamp'],unit='s').dt.strftime('%Y-%m-%d')

In [23]:
df_ratings['timestamp'] = pd.to_datetime(df_ratings['timestamp'])

In [24]:
df_ratings.head()

,userId,rating,timestamp,movieId
0,1,1.0,2015-03-09,as680
1,1,4.5,2015-03-09,ns2186
2,1,5.0,2015-03-09,hs2381
3,1,5.0,2015-03-09,ns3663
4,1,5.0,2015-03-09,as9500


In [25]:
print(df_ratings.dtypes)

userId                int64
rating              float64
timestamp    datetime64[ns]
movieId              object
dtype: object


In [26]:
# Renombramos la columna rating a score

df_ratings.rename(columns = {'rating':'score'}, inplace = True)

In [27]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024289 entries, 0 to 1499999
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   userId     int64         
 1   score      float64       
 2   timestamp  datetime64[ns]
 3   movieId    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 420.5+ MB


In [28]:
# Guardamos el dataset unificado

df_ratings.to_csv('datasets/scores.csv', index=False)

### Promedio de scores

In [11]:
df_movies = pd.read_csv('datasets/movies_titles.csv')

df_ratings = pd.read_csv('datasets/scores.csv')

prom_scores = df_ratings.groupby("movieId")
prom_scores = prom_scores["score"].mean()

prom_scores


movieId
as1       3.467131
as10      3.439571
as100     3.609302
as1000    3.556701
as1001    3.585288
            ...   
ns995     3.515625
ns996     3.626518
ns997     3.530526
ns998     3.582645
ns999     3.494781
Name: score, Length: 22998, dtype: float64

In [12]:
df_movies = df_movies.merge(prom_scores, left_on="id", right_on="movieId")

In [14]:
df_movies.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,platform,duration_int,duration_type,score
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,as1,amazon,113.0,min,3.467131
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,as2,amazon,110.0,min,3.548682
2,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74 min,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3,amazon,74.0,min,3.500000
3,s4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,69 min,documentary,"pink breaks the mold once again, bringing her ...",as4,amazon,69.0,min,3.538055
4,s5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,45 min,"drama, fantasy",teenage matt banting wants to work with a famo...,as5,amazon,45.0,min,3.478992


In [17]:
df_movies.to_csv("datasets/movies_titles.csv", index=False)